# Lectures on Machine Learning for Strong Gravity
## Lecture 3: Learning Orbital Dynamics

To test it, simply press Ctrl+Enter sequentially in each cell, or click on the small icons on the left with the "play" symbol.

<br>

<a href="https://colab.research.google.com/github/raimonluna/MachineLearningForStrongGravity/blob/main/Lecture3_Learning_Orbital_Dynamics.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### In this lecture, you will learn:
 - How we can learn the corrections to the Newtonian equations of motion by analyzing gravitational waves.
 
This method was introduced in:

 - https://arxiv.org/abs/2102.12695

<br>

In [ ]:
!pip install torchdiffeq

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

from torchdiffeq import odeint

In [ ]:
def SchwGeo(t, y):
    phi = y[:,0]
    chi = y[:,1]

    dphi = (p-2-2*e*torch.cos(chi)) * (1+e*torch.cos(chi))**2 / (M * p**1.5 * torch.sqrt((p-2)**2 - 4*e**2))
    dchi = dphi * torch.sqrt(p - 6 - 2*e*torch.cos(chi)) / torch.sqrt(p)

    return torch.stack((dphi, dchi), 1)

def NewtGeo(t, y):
    phi = y[:,0]
    chi = y[:,1]

    dphi = (1 + e*torch.cos(chi))**2 / (M * p**1.5)
    dchi = dphi
    
    return torch.stack((dphi, dchi), 1)

def waveform(y, dy, ddy):
    phi   = y[:,0]
    chi   = y[:,1]
    dphi  = dy[:,0]
    dchi  = dy[:,1]
    ddphi = ddy[:,0]
    ddchi = ddy[:,1]
    
    r = p * M / (1 + e * torch.cos(chi))
    x = r * torch.cos(phi)
    y = r * torch.sin(phi)
    
    dx = -((dphi*M*p*torch.sin(phi))/(1 + e*torch.cos(chi))) + (dchi*e*M*p*torch.cos(phi)*torch.sin(chi))/(1 + e*torch.cos(chi))**2
    dy = (M*p*(dphi*torch.cos(phi)*(1 + e*torch.cos(chi)) + dchi*e*torch.sin(phi)*torch.sin(chi)))/(1 + e*torch.cos(chi))**2
    
    ddx = -((M*p*(dphi**2*torch.cos(phi)*(1 + e*torch.cos(chi))**2 + (dchi**2*e*torch.cos(phi)*(-2*torch.cos(chi) + e*(-3 + torch.cos(2*chi))))/2. + 2*dphi*dchi*e*(1 + e*torch.cos(chi))*torch.sin(phi)*torch.sin(chi) + (1 + e*torch.cos(chi))*(ddphi*(1 + e*torch.cos(chi))*torch.sin(phi) - ddchi*e*torch.cos(phi)*torch.sin(chi))))/(1 + e*torch.cos(chi))**3)
    ddy = (M*p*(-(dphi**2*(1 + e*torch.cos(chi))**2*torch.sin(phi)) + (dchi**2*e*(3*e + 2*torch.cos(chi) - e*torch.cos(2*chi))*torch.sin(phi))/2. + 2*dphi*dchi*e*torch.cos(phi)*(1 + e*torch.cos(chi))*torch.sin(chi) + (1 + e*torch.cos(chi))*(ddphi*torch.cos(phi)*(1 + e*torch.cos(chi)) + ddchi*e*torch.sin(phi)*torch.sin(chi))))/(1 + e*torch.cos(chi))**3
    
    ddIxx =  (2/3) * (2 * dx**2 - dy**2 + 2 * x*ddx - y*ddy)
    ddIyy = -(2/3) * (dx**2 - 2 * dy**2 + x*ddx - 2 * y*ddy)
    
    return np.sqrt(4*np.pi/5) * (ddIxx - ddIyy)
    

In [ ]:
class NetGeo(nn.Module):
    def __init__(self):
        super(NetGeo, self).__init__()
        
        self.net = nn.Sequential(
            nn.Linear(1, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 2),
        )

        for m in self.net.modules():
            if isinstance(m, nn.Linear):
                nn.init.constant_(m.weight,val=0)
                nn.init.constant_(m.bias, val=0)
        
    def forward(self, t, y):
        yc = torch.cos(y[:,[1]])
        return NewtGeo(t, y) * (1 + self.net(yc))


class GetOrbit(nn.Module):
    def __init__(self, func):
        super(GetOrbit, self).__init__()
        self.func = func
    
    def forward(self, t, y):
        y = odeint(self.func, y0, t, method='dopri5').reshape(t.size()[0],2).to(device)
        phi = y[:,0]
        chi = y[:,1]
        r   = p * M / (1 + e * torch.cos(chi))
        
        return torch.stack((r * torch.cos(phi), r * torch.sin(phi)), 1)

class GetWaveform(nn.Module):
    def __init__(self, func):
        super(GetWaveform, self).__init__()
        self.func = func
    
    def forward(self, t, y):
        h = t[1] - t[0]
        y = odeint(self.func, y0, t, method='dopri5').reshape(t.size()[0],2).to(device)
        dy = self.func(t, y)
        ddy = (torch.roll(dy, -1, dims=0) - torch.roll(dy, 1, dims=0)) / (2*h)
        ddy[0]  = (dy[1] - dy[0]) / h
        ddy[-1] = (dy[-1] - dy[-2]) / h
        
        return  waveform(y, dy, ddy)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

p  = torch.tensor(100., device = device)
e  = torch.tensor(0.5, device = device)
M  = torch.tensor(1., device = device)

y0 = torch.tensor([[0., np.pi]], device = device)
t_train  = torch.linspace(0., 50000., 1000, device = device)

true_y_fun     = GetWaveform(SchwGeo)
true_orbit_fun = GetOrbit(SchwGeo)
true_y         = true_y_fun(t_train, y0)

netGeo         = NetGeo().to(device)
pred_y_fun     = GetWaveform(netGeo)
pred_orbit_fun = GetOrbit(netGeo)

optimizer = optim.Adam(netGeo.parameters(), lr=1e-3)

In [ ]:
for epoch in range(100):
    try:
        optimizer.zero_grad()

        pred_y = pred_y_fun(t_train, y0)
        loss = torch.sum((pred_y - true_y)**2)
        print(f'Training epoch {epoch} loss: {loss.item():.5f}')

        loss.backward()
        optimizer.step()
    except KeyboardInterrupt:
        break

In [ ]:
t_test  = torch.linspace(0., 300000., 6000, device = device)

true_y_train = true_y_fun(t_train, y0).cpu().detach().numpy()
true_y_test  = true_y_fun(t_test, y0).cpu().detach().numpy()
pred_y_test  = pred_y_fun(t_test, y0).cpu().detach().numpy()

true_orbit_train = true_orbit_fun(t_train, y0).cpu().detach().numpy()
true_orbit_test  = true_orbit_fun(t_test, y0).cpu().detach().numpy()
pred_orbit_test  = pred_orbit_fun(t_test, y0).cpu().detach().numpy()

hm_side = 100
test_ang = np.linspace(0, 2*np.pi, hm_side)
test_phi, test_chi = np.meshgrid(test_ang, test_ang)
hm_angles  = torch.tensor(np.concatenate([[test_phi.reshape(hm_side**2)], \
                                          [test_chi.reshape(hm_side**2)]]).transpose().astype('float32')).to(device)
hm_times   = torch.zeros(hm_side**2).to(device)
true_eom   = SchwGeo(hm_times, hm_angles)
learnt_eom = netGeo(hm_times, hm_angles)
rel_err    = (learnt_eom - true_eom) / true_eom

phi_dot    = rel_err[:,0].cpu().detach().numpy().reshape(hm_side,hm_side)
chi_dot    = rel_err[:,1].cpu().detach().numpy().reshape(hm_side,hm_side)

In [ ]:
# Plot figure with subplots of different sizes
fig = plt.figure(figsize = (10, 10))
font = {'size'   : 16}
plt.rc('font', **font)
# set up subplot grid
gridspec.GridSpec(3,3)

# Orbits subplot
plt.subplot2grid((3,3), (0,0), colspan=2, rowspan=2)
plt.plot(true_orbit_train[:,0], true_orbit_train[:,1], '.', color = 'black')
plt.plot(true_orbit_test[:,0], true_orbit_test[:,1], color = 'blue')
plt.plot(pred_orbit_test[:,0], pred_orbit_test[:,1], '--', color = 'red')
plt.xlabel('$x$')
plt.ylabel('$y$')

# Waveform subplot
plt.subplot2grid((3,3), (2,0), colspan=3, rowspan=1)
plt.plot(t_train.cpu().detach().numpy(), true_y_train, '.', color = 'black')
plt.plot(t_test.cpu().detach().numpy(), true_y_test, color = 'blue')
plt.plot(t_test.cpu().detach().numpy(), pred_y_test, '--', color = 'red')
plt.xlabel('$t$')
plt.ylabel('$\Re(h^{22})$')

# Error in phi
plt.subplot2grid((3,3), (0,2))
plt.title('Relative error in $\dot \phi$')
plt.pcolormesh(test_phi, test_chi, phi_dot, cmap='RdBu', shading='auto')
plt.colorbar()
plt.xlabel('$\phi$')
plt.ylabel('$\chi$')

# Error in chi
plt.subplot2grid((3,3), (1,2))
plt.title('Relative error in $\dot \chi$')
plt.pcolormesh(test_phi, test_chi, chi_dot, cmap='RdBu', shading='auto')
plt.colorbar()
plt.xlabel('$\phi$')
plt.ylabel('$\chi$')

fig.tight_layout()